<a href="https://colab.research.google.com/github/MohammedKQ/Bank-Loan-Classification/blob/main/Words_War.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install wordfreq deep-translator

import string
import random
from wordfreq import top_n_list
from deep_translator import GoogleTranslator

# ===== Build dictionary using wordfreq =====
raw_words = top_n_list("en", 10000)

# Keep only clean a-z words (no hyphens, apostrophes, accents, etc.)
web2lowerset = set(
    w for w in raw_words
    if w and w.isalpha() and w.islower() and all(ch in string.ascii_lowercase for ch in w)
)


In [ ]:

# ===== Alphabet values from 1–26 =====
alphabet = {letter: index + 1 for index, letter in enumerate(string.ascii_lowercase)}

# ===== Translation + Validation =====
_translation_cache = {}

def translate_to_arabic_cached(word):
    word = word.lower()
    if word in _translation_cache:
        return _translation_cache[word]
    try:
        ar = GoogleTranslator(source="en", target="ar").translate(word)
    except:
        ar = None
    _translation_cache[word] = ar
    return ar

# Uni Block for arabic
def contains_arabic(text):
    return any('\u0600' <= ch <= '\u06FF' for ch in text)

def is_good_arabic_translation(word, ar_text):
    if not ar_text or ar_text.strip() == "":
        return False
    ar_text = ar_text.strip()

    # If translation returns the same English word, reject
    if ar_text.lower() == word.lower():
        return False

    # Must contain Arabic characters
    if not contains_arabic(ar_text):
        return False

    return True


# ===== Game Class =====
class Game():

    def __init__(self):
        self.total_score_p1 = 0
        self.total_score_p2 = 0

    def play_round(self, word1, word2):
        # Normalize words to lowercase
        word1 = word1.lower().strip()
        word2 = word2.lower().strip()

        # ===== Rules =====

        # No numbers or symbols allowed
        if not word1.isalpha() or not word2.isalpha():
            print("No numbers or symbols are allowed")
            return

        # Words must have the same length
        if len(word1) != len(word2):
            print("Words must be same length\n")
            return

        # Must be a full word, not a single letter
        if len(word1) == 1:
            print("Enter a word not a letter\n")
            return

        # Words must exist in the dictionary
        if word1 not in web2lowerset or word2 not in web2lowerset:
            missing = []
            if word1 not in web2lowerset: missing.append(word1)
            if word2 not in web2lowerset: missing.append(word2)
            print("Not in dictionary:", ", ".join(missing), "\n")
            return

        # Reset round scores
        round_score1 = 0
        round_score2 = 0

        print("\n" + "="*30)
        print(f"  {word1}  VS  {word2}")
        print("="*30)

        for l1, l2 in zip(word1, word2):
            value1 = alphabet[l1]
            value2 = alphabet[l2]

            # Compare letters by alphabetical value
            if value1 > value2:
                round_score1 += value1
                print(f"{l1} ({value1}) > {l2} ({value2})  ➜  Player 1 +{value1}")

            elif value2 > value1:
                round_score2 += value2
                print(f"{l2} ({value2}) > {l1} ({value1})  ➜  Player 2 +{value2}")

            else:
                print(f"{l1} ({value1}) = {l2} ({value2})  ➜  No points")

        # Update total scores
        self.total_score_p1 += round_score1
        self.total_score_p2 += round_score2

        print("-"*30)
        print(f"Round Score → P1: {round_score1} | P2: {round_score2}")

        if round_score1 > round_score2:
            print("Round Winner: Player 1")
        elif round_score2 > round_score1:
            print("Round Winner: Player 2")
        else:
            print("Round Draw")

        print("TOTAL SCORE")
        print(f"Player 1: {self.total_score_p1}")
        print(f"Player 2: {self.total_score_p2}")
        print("="*39)

        # Arabic translations
        ar1 = translate_to_arabic_cached(word1)
        ar2 = translate_to_arabic_cached(word2)

        print("Arabic Translation:")
        print(f"{word1} → {ar1 if ar1 else 'Translation failed'}")
        print(f"{word2} → {ar2 if ar2 else 'Translation failed'}")
        print("="*30 + "\n")


    # ===== Play against AI =====
    def AI_OR_PVP(self, word1):

        word1 = word1.lower().strip()

        # Validate input
        if not word1.isalpha():
            print("No numbers or symbols are allowed")
            return

        if len(word1) == 1:
            print("Enter a word not a letter\n")
            return

        if word1 not in web2lowerset:
            print(f"{word1} not in dictionary\n")
            return

        # Filter words with same length (AI chooses from wordfreq list too)
        same_length_words = [w for w in web2lowerset if len(w) == len(word1)]

        if not same_length_words:
            print("No matching word length found for computer")
            return

        # Pick a word that has a good Arabic translation
        max_tries = 50
        ai_word = None

        for _ in range(max_tries):
            candidate = random.choice(same_length_words)
            ar = translate_to_arabic_cached(candidate)
            if is_good_arabic_translation(candidate, ar):
                ai_word = candidate
                break

        if ai_word is None:
            print("AI couldn't find a word with a good Arabic translation. Try again.")
            return

        print(f"\nComputer chose: {ai_word}")

        self.play_round(word1, ai_word)


# ===== Main Loop =====
game = Game()

while True:

    print("Enter 0 to exit")
    choose = input("Choose mode: 1) PVP 2) VS AI : ").strip()

    if choose == "0":
        print("Game Ended")
        break

    elif choose == "1":
        word1 = input("Enter Word 1: ").strip()
        if word1 == "0":
            print("Game Ended")
            break

        word2 = input("Enter Word 2: ").strip()
        if word2 == "0":
            print("Game Ended")
            break

        game.play_round(word1, word2)

    elif choose == "2":
        word1 = input("Enter word 1: ").strip()
        if word1 == "0":
            print("Game Ended")
            break

        game.AI_OR_PVP(word1)

    else:
        print("Invalid choice\n")

Enter 0 to exit
Choose mode: 1) PVP 2) VS AI : 0
Game Ended


Reflection

What was the most challenging part?

The most challenging aspects were handling invalid word inputs reliably, generating AI words of the same length while ensuring valid Arabic translations, and managing complex Streamlit behaviors such as reruns, session state consistency, and unexpected UI errors.


Which concept did we enjoy the most?

We enjoyed experimenting with different libraries and testing multiple approaches before selecting the most suitable one.
Playing the game with each other and challenging the AI to see if we could beat it.


What would we improve if we had more time?


With additional time, we would further enhance the Streamlit user interface to better align with the game logic and improve the overall user experience. We would also implement AI difficulty levels to make gameplay more dynamic, balanced, and strategically engaging.

In [ ]:
    """
    TODO:
    Need optmizing :DONE
    Need function for matching word : DONE
    Need function for Random word : DONE
    install new dict for explaining  mean of the word : DONE
    install new dict for translation purpose :  DONE
    Giving the choice to ask the user if he want to play vs bot or pvp : DONE
    Add an loop if the player chose Player vs Player so it ask for a new word or words DONE
    If a single letter is inputed the Translation should say letter not word .(Maybe rejcet a single letter ?) DONE
    Maybe sperate the player from the AI score
"""
